In [61]:
import pandas as pd
import numpy as np
import tensorflow as tf

path = "C:\\pytest\\감성대화말뭉치\\"
data = pd.read_excel(path+"감성대화말뭉치(최종데이터)_Training.xlsx")
random_state = 1111

# 연령, 감정 모델 각각 만들기
data = data[['연령', '감정_대분류', '사람문장1', '사람문장2', '사람문장3', '사람문장4']]
# data = data.fillna('')
data = data.rename(
    columns={'사람문장1': '사람문장', '사람문장2': '사람문장', '사람문장3': '사람문장', '사람문장4': '사람문장'})
col_data = pd.concat([data.iloc[:, [0, 1, 2]], data.iloc[:, [0, 1, 3]], data.iloc[:, [0, 1, 4]], data.iloc[:, [0, 1, 5]]], axis=0)
col_data.dropna(inplace=True)
col_data = col_data.reset_index()
col_data.drop('index', axis=1, inplace=True)


In [62]:
import rhinoMorph
rn = rhinoMorph.startRhino()

def morphed(text):
    tmp_list = rhinoMorph.onlyMorph_list(
        rn, text, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'MM', 'MAG', 'EF']) #ef, -ic
    return ' '.join(tmp_list)


filepath:  c:\projects\venv\lib\site-packages
classpath:  c:\projects\venv\lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


In [63]:
col_data['사람문장_형태소'] = col_data['사람문장'].apply(morphed)


In [64]:
col_data


,연령,감정_대분류,사람문장,사람문장_형태소
0,청년,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.,아내 드디어 출산 되 정말 신 나
1,노년,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,당뇨 합병증 약이 가지 넘어가 스트레스
2,청소년,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 올라오 중학교 때 수업 갑자기 어렵 당황
3,노년,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 되 받 되 첫 월급 오 가족 외식 하 예정 야 너무 행복
4,노년,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚 드디어 다 갚 되 이제야 안도감 들
...,...,...,...,...
114284,노년,불안,미리 미리 건강 챙기고 모두 안 아팠으면 좋겠어.,미리 미리 건강 챙기 모두 안 아프 좋 어
114285,노년,당황,주변에 믿음직한 사람들에게서 정보도 많이 얻고 달콤한 말은 항상 의심하고 볼래.,주변 믿음직 사람 정보 많이 얻 달콤 말 항상 의심 보 ㄹ래
114286,노년,불안,친구들에게 내 마음을 터놓고 얘기하면 좀 나아질 것 같아.,친구 마음 터놓 이야기 좀 낫 같
114287,노년,당황,남편에게 이런 내 마음을 솔직하게 얘기해 봐야겠어.,남편 이런 마음 솔직 이야기 어


In [65]:
from sklearn.preprocessing import LabelEncoder

e_age = LabelEncoder()
e_age.fit(col_data.iloc[:, 0])
y_age = e_age.transform(col_data.iloc[:, 0])

e_senti = LabelEncoder()
# {'분노': 0, '기쁨': 1, '슬픔': 2, '당황': 3, '상처': 4, '불안': 5, '기쁨 ': 6, '불안 ': 7}
tmp = col_data.iloc[:, 1]
tmp = tmp.replace('기쁨 ', '기쁨')
tmp = tmp.replace('불안 ', '불안')
e_senti.fit(tmp)
y_senti = e_senti.transform(tmp)

In [66]:
from sklearn.model_selection import train_test_split
연령_X_pre, 연령_X_test, 연령_y_pre, 연령_y_test = train_test_split(
    col_data.iloc[:, -1], y_age, random_state=random_state, test_size=.3)
연령_X_train, 연령_X_validation, 연령_y_train, 연령_y_validation = train_test_split(
    연령_X_pre, 연령_y_pre, random_state=random_state, test_size=.1)
# test 데이터는 앞에서 3000건만 취하고 나머지 버림
연령_X_test = 연령_X_test[:3000]
연령_y_test = 연령_y_test[:3000]

감정_X_pre, 감정_X_test, 감정_y_pre, 감정_y_test = train_test_split(
    col_data.iloc[:, -1], y_senti, random_state=random_state, test_size=.3)
감정_X_train, 감정_X_validation, 감정_y_train, 감정_y_validation = train_test_split(
    감정_X_pre, 감정_y_pre, random_state=random_state, test_size=.1)
# test 데이터는 앞에서 3000건만 취하고 나머지 버림
감정_X_test = 감정_X_test[:3000]
감정_y_test = 감정_y_test[:3000]


In [67]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import numpy as np
import math

In [68]:
import numpy as np
text_len = [len(line.split(' ')) for line in 연령_X_train]
print('최소길이: ', np.min(text_len))
print('최대길이: ', np.max(text_len))
print('평균길이: ', np.round(np.mean(text_len), 1))
print('중간길이: ', np.median(text_len))
print('구간길이: ', np.percentile(text_len, [0, 25, 50, 75, 90, 100]))

최소길이:  1
최대길이:  32
평균길이:  8.4
중간길이:  8.0
구간길이:  [ 1.  6.  8. 10. 12. 32.]


In [69]:
max_words = 3000
maxlen = 20

In [70]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(연령_X_train)
word_index_age = tokenizer.word_index

In [71]:
print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index_age))
print('word_index type: ', type(word_index_age))
# print('word_index: ', word_index)

전체에서 12139개의 고유한 토큰을 찾았습니다.
word_index type:  <class 'dict'>


In [72]:
data = tokenizer.texts_to_sequences(연령_X_train)

print('data 0:', data[1])
print('texts 0:', 연령_X_train.iloc[1])

data = pad_sequences(data, maxlen=maxlen)

print('data:', data)
print('data 0:', data[0])
print('data 0의 길이:', len(data[0]))

연령_X_train_data = pad_sequences(data, maxlen=maxlen)
print(연령_X_train_data[0])
print(len(연령_X_train_data[0]))

data 0: [3, 456, 257, 524, 1090, 12, 255, 28, 197, 7]
texts 0: 친구 생일 선물 예쁘 가방 다 바쁘 정말 고맙 있
data: [[   0    0    0 ...   27  174    7]
 [   0    0    0 ...   28  197    7]
 [   0    0    0 ...  302 1078   14]
 ...
 [   0    0    0 ...  111   29   10]
 [   0    0    0 ...    4   29   10]
 [   0    0    0 ...   79  921  188]]
data 0: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 1562 1994   27  174    7]
data 0의 길이: 20
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 1562 1994   27  174    7]
20


In [73]:
data = tokenizer.texts_to_sequences(연령_X_train)
# print("data:", data)
len_d= [len(d) for d in data]
# print("길이", len_d)
print("최대문장길이: ", max(len_d))
print("최소문장길이: ", min(len_d))

연령_X_train_data = pad_sequences(data, maxlen=maxlen)
print(연령_X_train_data[0])
print(len(연령_X_train_data[0]))

최대문장길이:  29
최소문장길이:  0
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 1562 1994   27  174    7]
20


In [74]:
data = tokenizer.texts_to_sequences(연령_X_validation)
# print("data:", data)
len_d= [len(d) for d in data]
# print("길이", len_d)
print("최대문장길이: ", max(len_d))
print("최소문장길이: ", min(len_d))

연령_X_validation_data = pad_sequences(data, maxlen=maxlen)
print(연령_X_validation_data[0])
print(len(연령_X_validation_data[0]))

최대문장길이:  24
최소문장길이:  1
[   0    0    0    0    0    0    0    0    0   62 1693   17  101   78
  169  255 1437  574   79    4]
20


In [75]:
data = tokenizer.texts_to_sequences(연령_X_test)
# print("data:", data)
len_d= [len(d) for d in data]
# print("길이", len_d)
print("최대문장길이: ", max(len_d))
print("최소문장길이: ", min(len_d))

연령_X_test_data = pad_sequences(data, maxlen=maxlen)
print(연령_X_test_data[0])
print(len(연령_X_test_data[0]))

최대문장길이:  28
최소문장길이:  1
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 2303 1384 1634    2]
20


In [76]:
data = tokenizer.texts_to_sequences(감정_X_train)
# print("data:", data)
len_d= [len(d) for d in data]
# print("길이", len_d)
print("최대문장길이: ", max(len_d))
print("최소문장길이: ", min(len_d))

감정_X_train_data = pad_sequences(data, maxlen=maxlen)
print(감정_X_train_data[0])
print(len(감정_X_train_data[0]))

최대문장길이:  29
최소문장길이:  0
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 1562 1994   27  174    7]
20


In [77]:
data = tokenizer.texts_to_sequences(감정_X_validation)
# print("data:", data)
len_d= [len(d) for d in data]
# print("길이", len_d)
print("최대문장길이: ", max(len_d))
print("최소문장길이: ", min(len_d))

감정_X_validation_data = pad_sequences(data, maxlen=maxlen)
print(감정_X_validation_data[0])
print(len(감정_X_validation_data[0]))

최대문장길이:  24
최소문장길이:  1
[   0    0    0    0    0    0    0    0    0   62 1693   17  101   78
  169  255 1437  574   79    4]
20


In [78]:
data = tokenizer.texts_to_sequences(감정_X_test)
# print("data:", data)
len_d= [len(d) for d in data]
# print("길이", len_d)
print("최대문장길이: ", max(len_d))
print("최소문장길이: ", min(len_d))

감정_X_test_data = pad_sequences(data, maxlen=maxlen)
print(감정_X_test_data[0])
print(len(감정_X_test_data[0]))

최대문장길이:  28
최소문장길이:  1
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 2303 1384 1634    2]
20


In [79]:
def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [80]:
class_number = 4

연령_y_train_one_hot_encoding = to_one_hot(연령_y_train, dimension=class_number)
연령_y_validation_one_hot_encoding = to_one_hot(연령_y_validation, dimension=class_number)
연령_y_test_one_hot_encoding = to_one_hot(연령_y_test, dimension=class_number)


In [81]:
감정_y_train

array([2, 0, 5, ..., 3, 3, 5])

In [82]:
class_number_senti = 6

감정_y_train_one_hot_encoding = to_one_hot(감정_y_train, dimension=class_number_senti)
감정_y_validation_one_hot_encoding = to_one_hot(감정_y_validation, dimension=class_number_senti)
감정_y_test_one_hot_encoding = to_one_hot(감정_y_test, dimension=class_number_senti)

In [83]:
연령_X_train_data = to_one_hot(연령_X_train_data, dimension=max_words)
연령_X_validation_data = to_one_hot(연령_X_validation_data, dimension=max_words)
연령_X_test_data = to_one_hot(연령_X_test_data, dimension=max_words)
감정_X_train_data = to_one_hot(감정_X_train_data, dimension=max_words)
감정_X_validation_data = to_one_hot(감정_X_validation_data, dimension=max_words)
감정_X_test_data = to_one_hot(감정_X_test_data, dimension=max_words)


In [85]:
epochs = 10
batch_size= 1
model_name= 'c:\\projects\\model\\221014\\train_data_morphed_age.h5'
tokenizer_name= 'c:\\projects\\model\\221014\\train_data_morphed_age.pickle'

from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(연령_X_train_data.shape[1],)))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=연령_y_train_one_hot_encoding.shape[1], activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics='acc')

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
cp_callback = ModelCheckpoint(filepath=model_name, monitor='val_loss', verbose=1, save_best_only=True)
history = model.fit(연령_X_train_data, 연령_y_train_one_hot_encoding, epochs=epochs, batch_size=batch_size, validation_data=(연령_X_validation_data, 연령_y_validation_one_hot_encoding), callbacks=[earlystop_callback, cp_callback], verbose=1)
history_dict = history.history


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                192064    
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 194,276
Trainable params: 194,276
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
72001/72001 [==============================] - ETA: 0s - loss: 1.2015 - acc: 0.5421
Epoch 1: val_loss improved from inf to 1.16236, saving model to c:\projects\model\221014\train_data_morphed_age.h5
72001/72001 [==============================] - 405s 6ms/step - loss: 1.2015 - acc: 0.5421 - val_loss: 

In [86]:
import pickle
with open(tokenizer_name, 'wb') as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

In [87]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Validation accuracy of each epoch:', np.round(val_acc, 3))
epochs = range(1, len(val_acc) + 1)

Validation accuracy of each epoch: [0.55  0.57  0.528 0.578]


In [88]:
from keras.models import load_model
loaded_model_age = load_model(model_name)

with open(tokenizer_name, 'rb') as handle:
    loaded_tokenizer_age = pickle.load(handle)

In [93]:
test_eval = loaded_model_age.evaluate(연령_X_test_data, 연령_y_test_one_hot_encoding)

94/94 [==============================] - 1s 4ms/step - loss: 1.2214 - acc: 0.5473
